In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from rich.console import Console
from rich.table import Table
from rich import box
from rich.text import Text

# Define the neural network architecture
class PINN(tf.keras.Model):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.hidden_layers = [tf.keras.layers.Dense(layer, activation='tanh') for layer in layers]
        self.output_layer = tf.keras.layers.Dense(2)  # Output: [u, v]

    def call(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)

# Define the loss functions
@tf.function
def loss_momentum(model, x, y, t):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, y, t])
        u_v = model(tf.concat([x, y, t], axis=1))
        u = u_v[:, 0]
        v = u_v[:, 1]
        u_x = tape.gradient(u, x)
        u_y = tape.gradient(u, y)
        u_t = tape.gradient(u, t)
        v_x = tape.gradient(v, x)
        v_y = tape.gradient(v, y)
        v_t = tape.gradient(v, t)
        u_xx = tape.gradient(u_x, x)
        u_yy = tape.gradient(u_y, y)
        v_xx = tape.gradient(v_x, x)
        v_yy = tape.gradient(v_y, y)
    del tape
    momentum_loss = u_t + u * u_x + v * u_y - (u_xx + u_yy) + v_t + u * v_x + v * v_y - (v_xx + v_yy)
    return tf.reduce_mean(tf.square(momentum_loss))

@tf.function
def loss_continuity(model, x, y, t):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, y, t])
        u_v = model(tf.concat([x, y, t], axis=1))
        u = u_v[:, 0]
        v = u_v[:, 1]
        u_x = tape.gradient(u, x)
        v_y = tape.gradient(v, y)
    del tape
    continuity_loss = u_x + v_y
    return tf.reduce_mean(tf.square(continuity_loss))

# Boundary and initial conditions for 2D parallel plates Poiseuille flow
@tf.function
def boundary_conditions(model, x, y, t):
    u_v = model(tf.concat([x, y, t], axis=1))
    u = u_v[:, 0]
    v = u_v[:, 1]
    bc_loss = tf.reduce_mean(tf.square(u - 1.0)) + tf.reduce_mean(tf.square(v))
    return bc_loss

# Training step
@tf.function
def train_step(model, optimizer, x, y, t):
    with tf.GradientTape() as tape:
        loss_mom = loss_momentum(model, x, y, t)
        loss_cont = loss_continuity(model, x, y, t)
        loss_bc = boundary_conditions(model, x, y, t)
        loss = loss_mom + loss_cont + loss_bc
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [2]:
import pandas as pd
import plotly.express as px

# Instantiate the model
layers = [20, 20, 20, 20]
model = PINN(layers)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Example data (to be replaced with actual data)
x = tf.random.uniform((100, 1), minval=0, maxval=1)
y = tf.random.uniform((100, 1), minval=0, maxval=1)
t = tf.random.uniform((100, 1), minval=0, maxval=1)

result_dict = {}

# Training loop
for epoch in range(1000):
    loss = train_step(model, optimizer, x, y, t)
    if epoch % 100 == 0:
        # print(f"Epoch: {epoch}, Loss: {loss}")
        result_dict[epoch] = loss.numpy()

result = pd.DataFrame({"epoch" : [k for k, v in result_dict.items()], "loss" : [v for k, v in result_dict.items()]})

print(result)

   epoch      loss
0      0  3.127211
1    100  0.020691
2    200  0.005979
3    300  0.002641
4    400  0.001430
5    500  0.000858
6    600  0.000563
7    700  0.000406
8    800  0.000319
9    900  0.000267


In [3]:
# Plotting with plotly
fig = px.line(result, x='epoch', y='loss', title='Training Loss over Epochs', markers=True)
fig.update_yaxes(type='log')
fig.show()

In [4]:
# Plotting the results
x_plot = np.linspace(0, 1, 100)
y_plot = np.linspace(0, 1, 100)
x_plot, y_plot = np.meshgrid(x_plot, y_plot)
t_plot = np.zeros_like(x_plot)

x_flat = x_plot.flatten().reshape(-1, 1)
y_flat = y_plot.flatten().reshape(-1, 1)
t_flat = t_plot.flatten().reshape(-1, 1)

u_v_pred = model(tf.concat([x_flat, y_flat, t_flat], axis=1))
u_pred = u_v_pred[:, 0].numpy().reshape(x_plot.shape)
v_pred = u_v_pred[:, 1].numpy().reshape(x_plot.shape)

fig = go.Figure()
fig.add_trace(go.Surface(z=u_pred, x=x_plot, y=y_plot, colorscale='Viridis', name='u'))
fig.add_trace(go.Surface(z=v_pred, x=x_plot, y=y_plot, colorscale='Viridis', name='v'))
fig.update_layout(title='Flow Field u and v', scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='u, v'))
fig.show()